# Importing the required libraries

In [3]:
import requests
import pandas as pd
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

# Extract Data through Open API KEY

In [ ]:
# API link
BASE_URL = "https://data.cityofnewyork.us/resource/erm2-nwe9.json"

# Time range: June 1, 2025 – Sept 30, 2025
START_DATETIME = "2025-06-01T00:00:00"
END_DATETIME   = "2025-08-31T23:59:59"

# To filter to specfic date range and Manhattan area only
WHERE_CLAUSE = (
    f"created_date between '{START_DATETIME}' and '{END_DATETIME}' "
    f"AND borough = 'MANHATTAN'"
)


In [ ]:
def fetch_311_manhattan_summer(limit=50000, max_rows=None):
    """
    Fetch NYC 311 data for Manhattan between June–Sept 2025
    using paging (limit/offset).
    
    limit: rows per API call (max 50000 per docs)
    max_rows: optional safety cap on total rows (None = fetch all)
    """
    all_rows = []
    offset = 0

    while True:
        params = {
            "$where": WHERE_CLAUSE,
            "$limit": limit,
            "$offset": offset,
            # only select key columns to keep payload smaller
            "$select": ",".join([
                "created_date",
                "closed_date",
                "agency",
                "complaint_type",
                "descriptor",
                "status",
                "incident_zip",
                "incident_address",
                "latitude",
                "longitude",
                "resolution_description"
            ]),
            "$order": "created_date ASC"
        }

        response = requests.get(BASE_URL, params=params)
        if response.status_code != 200:
            print("Error status:", response.status_code, response.text)
            break

        batch = response.json()

        # No more rows
        if not batch:
            print("No more rows returned. Stopping.")
            break

        all_rows.extend(batch)
        print(f"Fetched batch: {len(batch)} rows | Total so far: {len(all_rows)}")

        if max_rows is not None and len(all_rows) >= max_rows:
            print(f"Reached max_rows={max_rows}. Stopping.")
            all_rows = all_rows[:max_rows]
            break

        # Move to next page
        offset += limit

    return all_rows


In [ ]:
# Fetch all rows for Manhattan, June–Sept 2025
raw_rows = fetch_311_manhattan_summer(limit=50000)
complaints = pd.DataFrame(raw_rows)

Fetched batch: 50000 rows | Total so far: 50000
Fetched batch: 50000 rows | Total so far: 100000
Fetched batch: 50000 rows | Total so far: 150000
Fetched batch: 11198 rows | Total so far: 161198
No more rows returned. Stopping.


# Save Data

In [ ]:
complaints = pd.DataFrame(raw_rows)
complaints.to_csv('complaints_raw.csv', index=False)

### View Data

In [8]:
complaints.head()

,created_date,closed_date,agency,complaint_type,descriptor,status,incident_zip,incident_address,resolution_description,latitude,longitude
0,2025-06-01T00:00:05.000,2025-06-01T00:51:12.000,NYPD,Noise - Residential,Loud Music/Party,Closed,10009.0,EAST 6 STREET,The Police Department responded to the complai...,NaN,NaN
1,2025-06-01T00:01:11.000,2025-06-01T01:49:17.000,NYPD,Abandoned Vehicle,With License Plate,Closed,10028.0,310 EAST 83 STREET,The Police Department responded to the complai...,40.775695,-73.952648
2,2025-06-01T00:03:34.000,2025-06-01T01:39:31.000,NYPD,Noise - Commercial,Loud Talking,Closed,10036.0,416 WEST 46 STREET,The Police Department responded to the complai...,40.761300,-73.991521
3,2025-06-01T00:03:51.000,2025-06-01T00:30:33.000,NYPD,Noise - Residential,Loud Music/Party,Closed,10001.0,340 9 AVENUE,The Police Department responded to the complai...,40.750544,-73.998365
4,2025-06-01T00:05:17.000,2025-06-01T00:34:22.000,NYPD,Noise - Residential,Loud Music/Party,Closed,10016.0,245 EAST 30 STREET,The Police Department responded to the complai...,40.742434,-73.978080


# Connect to Snowflake and Upload data 

In [ ]:
# Connect to Snowflake and upload data to RAW_311 table
conn = snowflake.connector.connect(
    user="KANGAROO",
    account="azb79167",
    authenticator="externalbrowser",  
    warehouse="FIVETRAN_WAREHOUSE",
    database="MLDS430",
    schema="KANGAROO_FINAL",
    role="TRAINING_ROLE"
)

# Upload the DataFrame to Snowflake as RAW_311 table
success, nchunks, nrows, _ = write_pandas(
    conn=conn,
    df=complaints,
    table_name="RAW_311",
    auto_create_table=True,
    overwrite=True
)

# Close the connection
conn.close()